In [2]:
import sqlite3
import pandas as pd
conn = sqlite3.connect('../test_data/lw.db')

/tmp/ipykernel_178971/4076997674.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
# SQL query
# First, select all unique geometries where all s11 predictions are below desired s11 between freq min and max (inner)
# Second, find all frequencies for the unique geometries (outer) 
df = pd.read_sql_query("""
    select * from geometries
    where ("cpw_in [mm]", "feed_l [mm]", "patch_l [mm]", "Feed_W [mm]", "ground_w [mm]", "patch_ground_w [mm]", "patch_w [mm]") in (
        select "cpw_in [mm]", "feed_l [mm]", "patch_l [mm]", "Feed_W [mm]", "ground_w [mm]", "patch_ground_w [mm]", "patch_w [mm]"
        from geometries
        where "Freq [GHz]" > ?
        and "Freq [GHz]" < ?
        group by "cpw_in [mm]", "feed_l [mm]", "patch_l [mm]", "Feed_W [mm]", "ground_w [mm]", "patch_ground_w [mm]", "patch_w [mm]"
        having max(prediction) < ?
    )
""", con=conn, params=(12, 16, -10, ))

In [ ]:
df

In [ ]:
df.iloc[0]['performances']

In [ ]:
df2 = df[['performances']]

In [ ]:
df2.iloc[0]['performances']

In [ ]:
df2[['Freq [GHz]', 'prediction']] = df2['performances'].str.split(';', expand=True)



In [ ]:
df_freq = df2['Freq [GHz]'].str.split(',', expand=True).stack().reset_index(level=1, drop=True).rename('Freq [GHz]')
df_pred = df2['prediction'].str.split(',', expand=True).stack().reset_index(level=1, drop=True).rename('prediction')


In [ ]:
df_freq

In [ ]:
dfc = pd.DataFrame()
dfc['Freq [GHz]'] = df_freq
dfc['prediction'] = df_pred

In [ ]:
df3 = df.drop(columns=["Freq [GHz]", "prediction", "performances"])

In [ ]:
df_fin

In [ ]:
import numpy as np
df3['Geometry Name'] = range(0, len(df3))
df3['Geometry Name']
df_fin = pd.DataFrame(np.repeat(df3, 440, axis=0), columns=df3.columns)
df_fin["Freq [GHz]"] = df_freq.values
df_fin["prediction"] = df_pred.values

In [ ]:
df_fin

### Try to find all rows where geometries have all values below -10 across entire frequency range 
Then, count rows and compare to how many are in the regular .db file 

In [31]:
from contextlib import closing

In [44]:
with closing(conn.cursor()) as cur:
    result_lessthan10 = cur.execute("""
        select count(*) from geometries
        where ("cpw_in [mm]", "feed_l [mm]", "patch_l [mm]", "Feed_W [mm]", "ground_w [mm]", "patch_ground_w [mm]", "patch_w [mm]") in (
            select "cpw_in [mm]", "feed_l [mm]", "patch_l [mm]", "Feed_W [mm]", "ground_w [mm]", "patch_ground_w [mm]", "patch_w [mm]"
            from geometries
            group by "cpw_in [mm]", "feed_l [mm]", "patch_l [mm]", "Feed_W [mm]", "ground_w [mm]", "patch_ground_w [mm]", "patch_w [mm]"
            having min(prediction) > -8
        )
    """)
    count_lessthan10 = result_lessthan10.fetchone()

In [ ]:
with closing(conn.cursor()) as cur:
    result_total = cur.execute("select count(*) from geometries")
    count_total = result_total.fetchone()

In [ ]:
print(f"how many geometries where all are greater than -10: {count_lessthan10[0]} / {count_total[0]}")

how many geometries where all are greater than -10: 164630 / 251868750
